In [1]:
import pandas as pd
import numpy as np

In [2]:
tb_caged = pd.read_excel('3-tabelas.xlsx', 'Tabela 8')

C:\Users\kevyn\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [3]:
primeiro_mes = tb_caged.loc[3][4]

In [4]:
#removendo a coluna de variação, que tem uma unidade de medida diferente das outras numéricas
tb_caged.drop(columns=tb_caged.columns[(tb_caged == 'Variação Relativa (%)').any()], inplace = True)

In [5]:
tb_caged = tb_caged.drop(tb_caged.index[[0,1,2,3,4,5]])

In [6]:
tb_caged.drop(columns=['Unnamed: 0'], inplace = True) # removendo essa coluna que não tem informações
tb_caged.dropna(subset = ['Unnamed: 2'], inplace=True) # removendo todas as linhas em que o código do município está vazio

n = 6 # Quantidade de colunas a remover do final da tabela
tb_caged.drop(columns=tb_caged.columns[-n:], axis=1, inplace=True) # removendo as últimas n colunas, que não tratam dos dados históricos

In [7]:
mes = primeiro_mes[:-5]

if mes == 'Janeiro':
    mes_inicial = 1
elif mes == 'Fevereiro':
    mes_inicial = 2
elif mes == 'Março':
    mes_inicial = 3
elif mes == 'Abril':
    mes_inicial = 4
elif mes == 'Maio':
    mes_inicial = 5
elif mes == 'Junho':
    mes_inicial = 6
elif mes == 'Julho':
    mes_inicial = 7
elif mes == 'Agosto':
    mes_inicial = 8
elif mes == 'Setembro':
    mes_inicial = 9
elif mes == 'Outubro':
    mes_inicial = 10
elif mes == 'Novembro':
    mes_inicial = 11
elif mes == 'Dezembro':
    mes_inicial = 12

In [8]:
total_meses = int(((len(tb_caged.columns))-3)/4)
ano_inicial = int((primeiro_mes[len(primeiro_mes)-4:]))
ano_final = ano_inicial + int((total_meses/12))

column_movimentos = []

if mes_inicial != 1:
    for m in range(mes_inicial,12 +1):
        column_movimentos.append('Estoque-{}/{}'.format(m,ano_inicial))
        column_movimentos.append('Admissões-{}/{}'.format(m,ano_inicial))
        column_movimentos.append('Desligamentos-{}/{}'.format(m,ano_inicial))
        column_movimentos.append('Saldos-{}/{}'.format(m,ano_inicial))
        
    for a in range(ano_inicial+1,ano_final + 1):
        for m in range(1,12 +1):
            column_movimentos.append('Estoque-{}/{}'.format(m,a))
            column_movimentos.append('Admissões-{}/{}'.format(m,a))
            column_movimentos.append('Desligamentos-{}/{}'.format(m,a))
            column_movimentos.append('Saldos-{}/{}'.format(m,a))
else:
    for a in range(ano_inicial,ano_final + 1):
        for m in range(1,12 +1):
            column_movimentos.append('Estoque-{}/{}'.format(m,a))
            column_movimentos.append('Admissões-{}/{}'.format(m,a))
            column_movimentos.append('Desligamentos-{}/{}'.format(m,a))
            column_movimentos.append('Saldos-{}/{}'.format(m,a))

In [9]:
column_ids = ['UF', 'Código do Município', 'Município']
column_names = column_ids + column_movimentos[0:(total_meses)*4]

In [10]:
tb_caged.set_axis(column_names, axis=1,inplace=True)
tb_caged_transposed = tb_caged.melt(column_ids, var_name='Movimentação', value_name='Quantidade')
tb_caged_transposed[['Movimentação','Período']] = tb_caged_transposed['Movimentação'].str.split('-',expand=True)
tb_caged_transposed['Município'] = np.where(tb_caged_transposed['Município'] == 'Não identificado','Não identificado', tb_caged_transposed['Município'].str.slice(start=3))
tb_caged_transposed['Quantidade'] = pd.to_numeric(tb_caged_transposed['Quantidade'],errors='coerce',downcast='integer').fillna(0).astype(int)

In [11]:
tb_caged_transposed.head()

,UF,Código do Município,Município,Movimentação,Quantidade,Período
0,RO,110001,Alta Floresta D Oeste,Estoque,1631,1/2020
1,RO,110002,Ariquemes,Estoque,15002,1/2020
2,RO,110003,Cabixi,Estoque,403,1/2020
3,RO,110004,Cacoal,Estoque,15189,1/2020
4,RO,110005,Cerejeiras,Estoque,1830,1/2020


In [12]:
tb_caged_transposed.to_csv(r'tabela_8_normalizada.csv',sep = ';',encoding='utf-8-sig',index=False)

In [13]:
tb_caged_ajustada = tb_caged_transposed.pivot(index=['UF', 'Código do Município','Município', 'Período'], columns=["Movimentação"], values="Quantidade").reset_index()

In [14]:
tb_caged_ajustada = tb_caged_ajustada[['UF', 'Código do Município', 'Município', 'Período', 'Estoque', 'Admissões', 'Desligamentos', 'Saldos']]

In [15]:
tb_caged_ajustada

Movimentação,UF,Código do Município,Município,Período,Estoque,Admissões,Desligamentos,Saldos
0,AC,120001,Acrelandia,1/2020,1034,22,13,9
1,AC,120001,Acrelandia,1/2021,1037,26,19,7
2,AC,120001,Acrelandia,1/2022,1007,17,25,-8
3,AC,120001,Acrelandia,1/2023,999,28,45,-17
4,AC,120001,Acrelandia,10/2020,1036,24,15,9
...,...,...,...,...,...,...,...,...
228406,TO,172210,Xambioa,8/2021,1252,42,23,19
228407,TO,172210,Xambioa,8/2022,1376,69,40,29
228408,TO,172210,Xambioa,9/2020,1116,18,11,7
228409,TO,172210,Xambioa,9/2021,1252,32,32,0


In [16]:
tb_caged_ajustada.to_csv(r'tabela_8_ajustada.csv',sep = ';',encoding='utf-8-sig',index=False)